In [ ]:
#To install NLTK use command
    #!pip install nltk
#Install textblob using:
    #!pip install textblob

In [2]:
from indicnlp.tokenize import sentence_tokenize
import pandas as pd
from textblob import TextBlob
from inltk.inltk import remove_foreign_languages
from csv import writer
import pickle
from difflib import get_close_matches
from difflib import SequenceMatcher

In [12]:


def malayalam_spellcheck():
    print("Working started")
    #Vocabulary dataset and convert it into list
    vocab=pd.read_csv('dataset/mlvocabe.csv',encoding="utf-16LE",header=None)
    vocablist=vocab[0].tolist()
    
    
    #Get the training dataset for spell correction.
    import pathlib
    import codecs

    docs=[]
    for flist in  pathlib.Path('dataset/train').iterdir():
        #open each files in directory
        if flist.is_file():
            for line in codecs.open(flist, encoding='utf-8'):
                #Append each lines in a file to th list doc[]
                docs.append(line.strip())

    print("Length of doc[]=",len(docs))
    
    #Tokenize each words in the list  doc[] using word_tokenize in nltk.tokenize
    from nltk.tokenize import word_tokenize
    voc2=[]
    for d in docs:
        toklist=word_tokenize(d)
        for token in toklist:
            #Append unique words to the list voc2[]
            if token not in voc2:
                voc2.append(token)
    print("Length of voc2[]=",len(voc2))
    
    
    Save the word tokens in to a file using pickle module for future use
    
    Create new pickle file in write binary mode
    with open('trainset.pkl', 'wb') as f:
        # Dump the data in list voc2 to the file trainingset.pkl
        pickle.dump(voc2, f)
    print("Data pickled successfully")

    print("Reading input data\n")
    #Read the input file for spell correction
    iput=open('C:/Users/Janeeta/Documents/NLP_Work/Indian_SpellChecker/inputdata.txt','r',encoding="utf-8")
    test_data=iput.read()
    print("Input data:\n",test_data)
    
    #Function to store the word,its validity and suggestions if the word is invalid, to the output dataset
    
    def updatetocsv(store_data):

        data = [store_data]
        dataframe = pd.DataFrame(data)
        dataframe.to_csv("dataset/mloutput/out.csv", mode='a',sep='\t',header=None)
        
    #Convert the input file text data in to text blob
    test_data=TextBlob(test_data)

    #Split the text in to words
    input1=test_data.words

    dict={}
    inc=[]
    print("\n\nChecking invalid words:\n")
    for words in input1:
            #If the word is in vocablist put the word in to dict{} as the key and 'correct' as value othrwise the value will be 'Incorrect'
            if words in vocablist:
                dict[words]="correct"
                #function call to update the csv file with new word
                updatetocsv([words,'Valid',None,None,None])
            else:
                dict[words]="Incorrect"
                # Append incorrect words to the list inc[]
                inc.append(words)

    print("Words and validity:",dict)            
        
        
    #Un pickle the trained dataset to get the possible suggestions of invalid words
    
    with open('trainset.pkl', 'rb') as f:
        trainset = pickle.load(f)
    
    #Get the close matches for invalid words using difflib Library
    
    from difflib import get_close_matches

    sugg_list={}
    for word in inc:
        #Get close 3 matches for each word
        gm=get_close_matches(word,trainset,3)
        #Store word and suggestions to dictionary 
        sugg_list[word]=gm
        
        
    from difflib import SequenceMatcher

    #a and b are the words to check for similarity ratio.
    #ratio() function for get the similarity between 0 to 1.Lower value indicates lower similarity and higher value indicates higher similarity
    def similar(a, b):
        return SequenceMatcher(None, a, b).ratio()
    
    #Find out the similarity between invalid words and each word suggestions and update the data into output dataset
    

    confdict={}
    print("word suggestions for incorrect words:\n")
    for word,suggs in sugg_list.items():
        print(word)
        valid_sugg_list=[word,'invalid']

        #For each word in suggestion list, find out the similarity ratio between each invalid word and its suggestions 
        for x in suggs:

            conf=similar(word,x)
            confdict[x]=conf
            valid_sugg_list.append(x)

        print("Valid suggestions:\n",confdict)        

        updatetocsv(valid_sugg_list)
        confdict.clear()
    
    
    
    #Print the output
    out= pd.read_csv("dataset/mloutput/out.csv", sep='\t',header=None,error_bad_lines=False)
    out=pd.DataFrame(out)
    print(out.iloc[:,1:])

In [ ]:

def hindi_spellcheck():
    print("Working started")


    def updatetocsv(store_data):
    
        data = [store_data]
        dataframe = pd.DataFrame(data)
        dataframe.to_csv("dataset/hindioutput/out.csv", mode='a',sep='\t',header=None)
        
        
    stopwords=open('dataset/hnstopwords.txt','r',encoding='utf-8')   
    sw=[]
    for x in stopwords:
        words=x.split(',')
        for y in words:
            y=y[2:-1]
            sw.append(y)
    print("Stopwords:\n",sw)
        
        
        
    def cleandata(d1,listdata):
        if d1 in sw:
            listdata.remove(d1)
        if d1=='':
            listdata.remove(d1)
        
    print("Reading input data")
    #Input test data
    iput=open('C:/Users/Janeeta/Documents/NLP_Work/Indian_SpellChecker/hndinput.txt','r',encoding="utf-8")
    test_data=iput.read()
    print("Input data:\n",test_data)
    
    
    vocab=pd.read_csv('dataset/hnvocabe.csv',encoding="utf-8",header=None)
    vocablist=vocab[0].tolist()
        

    #Convert the input file text data in to text blob
    test_data=TextBlob(test_data)
    
    #Split the text in to words
    input1=test_data.words
    
    dict={}
    inc=[]
    
    print("Checking validity of words")
    for words in input1:
            #If the word is in vocablist put the word in to dict{} as the key and 'correct' as value othrwise the value will be 'Incorrect'
            if words in vocablist:
                dict[words]="correct"
                #function call to update the csv file with new word
                updatetocsv([words,'Valid',None,None,None])
            else:
                dict[words]="Incorrect"
                # Append incorrect words to the list inc[]
                inc.append(words)
                
    print("Words and validity:\n",dict) 
    
    
    ##Train dataset
    
    train=pd.read_csv('dataset/hinditrain/train.csv',encoding="utf-8",header=None)
    train=train.iloc[1:,0]
    traindata=train.tolist()
    traindata=[remove_foreign_languages(train, 'hi') for train in traindata]
    
    trainset=[]
    for pgf in traindata:
        for x in pgf:
            trainset.append(x[1:])
        
    for pg in trainset[1:10000]:
        cleandata(pg,trainset)
    
    len(trainset)
    
    ##Save the trained list as a file
    

    #Create new pickle file in write binary mode
    with open('hntrainingset.pkl', 'wb') as f:
        # Dump the data in list voc2 to the file trainingset.pkl
        pickle.dump(trainset, f)
    print("Data pickled successfully")

    with open('hntrainingset.pkl', 'rb') as f:
        trainset = pickle.load(f)
   
    sugg_list={}
    for word in inc:
        #Get close 3 matches for each word
        gm=get_close_matches(word,trainset,3)
        #Store word and suggestions to dictionary 
        sugg_list[word]=gm

    
    #a and b are the words to check for similarity ratio.
    #ratio() function for get the similarity between 0 to 1.Lower value indicates lower similarity and higher value indicates higher similarity
    def similar(a, b):
        return SequenceMatcher(None, a, b).ratio()
    
    confdict={}
    print("word suggestions for incorrect words:\n")
    for word,suggs in sugg_list.items():
        print(word)
        valid_sugg_list=[word,'invalid']
        
        #For each word in suggestion list, find out the similarity ratio between each invalid word and its suggestions 
        for x in suggs:
     
            conf=similar(word,x)
            confdict[x]=conf
            valid_sugg_list.append(x)
    
        print("Valid suggestions:\n",confdict)        
        
        updatetocsv(valid_sugg_list)
        confdict.clear()    
    
    
    
    out= pd.read_csv("dataset/hindioutput/out.csv", sep='\t',header=None,error_bad_lines=False)
    out=pd.DataFrame(out)
    print(out.iloc[:,1:])



In [15]:
lang=int(input("Language Options:\n1.Malayalam\n2.Hindi\nEnter your option number:"))

if lang==1:
    malayalam_spellcheck()
elif lang==2:
    hindi_spellcheck()
else:
    print("Invalid option")

Language Options:
1.Malayalam
2.Hindi
Enter your option number:1
Working started
Length of doc[]= 34438
Reading input data

Input data:
 മനുഷ്യ സമുദായത്തിന്റെ ജന്മസിദ്ധമായ അന്തസ്സും സമാവകാശവും ലോകത്തിൽ സ്വാതന്ത്ര്യം, നീതി, സമാധാനം എന്നിവയുടെ സ്ഥാപനത്തിന്നു അടിസ്ഥാനമായിരിക്കുന്നതിനാലും മനുഷ്യാവകാശങ്ങളെ വകവെക്കാത്തതുകൊണ്ടു മനം മടുപ്പിക്കുന്ന ക്രൂര സംഭവങ്ങളുണ്ടാകുന്നതിനാലും സർവ്വതോന്മുഖമായ സ്വാതന്ത്ര്യവും സമൃദ്ധിയും മനുഷ്യനു അനുഭവിക്കാവുന്ന ഒരു പുതു ലോകത്തിന്റെ സ്ഥാപനമാണ്‌ പൊതുജനങ്ങളുടെ ആഗ്രഹം എന്ന് പ്രഖ്യാപിക്കപ്പെട്ടിരിക്കുന്നതിനാലും ഹിംസാമാർഗ്ഗം സ്വീകരിക്കാതിരിക്കണമെങ്കിൽ മനുഷ്യാവകാശങ്ങളെ നിയമാനുസൃതമായി വകവെച്ചു കൊടുക്കേണ്ടതാണെന്നുള്ളതിനാലും രാഷ്ട്രങ്ങൾ തമ്മിൽ സൌഹൃദം പുലർത്തേണ്ടതാണെന്നുള്ളതിനാലും ഐക്യരാഷ്ട്ര ജനത അവരുടെ കരാറിൽ സ്ത്രീപുരുഷ സമത്വത്തെക്കുറിച്ചും മനുഷ്യരുടെ മൌലികാവകാശത്തെക്കുറിച്ചും ജീവിതരീതി നന്നാക്കുന്നതിനെക്കുറിച്ചും ഒന്നുകൂടി ഊന്നിപ്പറഞ്ഞിരിക്കുന്നതിനാലും മനുഷ്യാവകാശങ്ങളേയും മൌലിക സ്വാതന്ത്ര്യത്തേയും അന്യോന്യം ബഹുമാനിച്ചുകൊള്ളാമെന്ന് ഐക്യരാഷ്ട്ര സമിതിയിലെ അംഗങ്ങൾ വാഗ്ദാ

പ്രഖ്യാപിക്കപ്പെട്ടിരിക്കുന്നതിനാലും
Valid suggestions:
 {'പ്രഖ്യാപിക്കപ്പെട്ടിരിക്കുന്നു': 0.9090909090909091, 'പ്രഖ്യാപിക്കപ്പെട്ടിരുന്നു': 0.8387096774193549, 'പരാമർശിക്കപ്പെട്ടിരിക്കുന്നു': 0.78125}
ഹിംസാമാർഗ്ഗം
Valid suggestions:
 {'ഗുഹാമാർഗ്ഗം': 0.782608695652174, 'വിമാനമാർഗ്ഗം': 0.75, 'മാർഗ്ഗം': 0.7368421052631579}
സ്വീകരിക്കാതിരിക്കണമെങ്കിൽ
Valid suggestions:
 {'കാണിക്കണമെങ്കിൽ': 0.6829268292682927, 'സ്വീകരിച്ചിരിക്കുന്ന': 0.6521739130434783, 'സ്വീകരിക്കപ്പെട്ടിരിക്കുന്ന': 0.6415094339622641}
നിയമാനുസൃതമായി
Valid suggestions:
 {'നിയമാനുസൃതമായ': 0.9629629629629629, 'നിയമാനുസൃതം': 0.8, 'അനുസൃതമായി': 0.75}
വകവെച്ചു
Valid suggestions:
 {'വെച്ചു': 0.8571428571428571, 'വെച്ചും': 0.8, 'പകരംവെച്ചു': 0.7777777777777778}
കൊടുക്കേണ്ടതാണെന്നുള്ളതിനാലും
Valid suggestions:
 {'നോക്കേണ്ടതാണെന്ന്': 0.6521739130434783, 'കൊടുക്കുന്നതിനു': 0.6363636363636364, 'ഉപയോഗിക്കേണ്ടതാണെന്നും': 0.6274509803921569}
രാഷ്ട്രങ്ങൾ
Valid suggestions:
 {'രാഷ്ട്രങ്ങൾ': 1.0, 'രാഷ്ടങ്ങൾ': 0.9, 'സഖ്യരാഷ്ട്രങ്ങൾ': 0.84

എന്നിവയെ
Valid suggestions:
 {'എന്നിവയെ': 1.0, 'എന്നിവയുടെ': 0.8888888888888888, 'എന്നിവയിലെ': 0.8888888888888888}
കണക്കാക്കാതെ
Valid suggestions:
 {'കണക്കാക്കാതെ': 1.0, 'കണക്കാക്കാം': 0.8695652173913043, 'കണക്കാക്കി': 0.8181818181818182}
പ്രഖ്യാപനത്തിൽ
Valid suggestions:
 {'പ്രസ്ഥാനത്തിൽ': 0.8148148148148148, 'പര്യടനത്തിൽ': 0.8, 'പ്രഖ്യാപനത്തിലുണ്ട്': 0.7878787878787878}
പറയുന്ന
Valid suggestions:
 {'പറയുന്ന': 1.0, 'പറയുന്നു': 0.9333333333333333, 'പറയുന്നു\u200c': 0.875}
അവകാശങ്ങൾക്കും
Valid suggestions:
 {'അവയവങ്ങൾക്കും': 0.8148148148148148, 'അവകാശങ്ങൾക്കായുള്ള': 0.8125, 'കാലങ്ങൾക്കു': 0.8}
സ്വാതന്ത്ര്യത്തിനും
Valid suggestions:
 {'സ്വാതന്ത്ര്യത്തിനു': 0.972972972972973, 'സ്വാതന്ത്ര്യത്തിലും': 0.9473684210526315, 'സ്വാതന്ത്ര്യത്തിന്': 0.918918918918919}
സർവ്വജനങ്ങളും
Valid suggestions:
 {'സർവ്വജനങ്ങളും': 1.0, 'സ്വപ്നങ്ങളും': 0.8, 'സേവനങ്ങളും': 0.782608695652174}
അർഹരാണ്‌
Valid suggestions:
 {'അർഹരാണ്': 0.9333333333333333, 'പേരാണ്\u200c': 0.6666666666666666, 'അർഥമാണ്': 0.6666666666666